<a href="https://colab.research.google.com/github/gillesvtsilvano/GANs/blob/master/GANS_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse exemplo segue o tutorial que pode ser encontrado [aqui](https://hub.packtpub.com/generative-adversarial-networks-using-keras/).

Para trabalhos futuros, observar:
* [Step-by-step](https://www.kdnuggets.com/2017/10/seven-steps-deep-learning-keras.html).
* Repositório do [Keras implementations of Generative Adversarial Networks](https://github.com/eriklindernoren/Keras-GAN)

# Introdução

O dataset utilizado será o MNIST (Modified National Institute of Standards and Technology database), dataset clássico contendo 60,000 caracteres números (0-9) escritos à mão. Esse dataset não oferece nenhum desafio (já foram superados), porém, é um ótimo exemplo para prova de conceito ou para aprender alguma conteúdo relacionado ao aprendizado de máquina, nesse caso,  Generative Adversarial Networks (GANs).

In [1]:
from keras.datasets import mnist #Loading MNIST dataset

Using TensorFlow backend.


<span style="color: red">TODO</span>
As GANs são compostas por duas redes neurais, uma Geradora (Generative) e outra Descritiva, que jogam _minmax_ uma contra a outra (Adversarial), no intuito de que a Descritiva evolua com base nos exemplos gerados pela Geradora, que aprenderá cada vez mais a gerar melhores exemplos, melhorando a qualidade do aprendizado da Descritiva.

